In [1]:
!export LC_ALL=en_US.UTF-8
!CUDA_VISIBLE_DEVICES=0,1,2,3
from embedding import *

In [2]:
import json
from pprint import pprint
import re
import numpy as np

import torch
try:
    import nltk
except:
    !pip install nltk
    import nltk
    nltk.download('punkt')
    nltk.download('stopwords')

In [3]:
data = json.load(open('../train-v1.1.json'))

In [4]:
from nltk.corpus import stopwords
nltk.download('stopwords')
nltk.download('punkt')
nltkStopWords = stopwords.words('english')
punctuations = [',', '?', '.', '-',]

[nltk_data] Downloading package stopwords to /home/ra2630/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /home/ra2630/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [5]:
def extractor(data):
    contexts = []
    qas = []
    for i in range(len(data["data"])):
        for j in range(len(data["data"][i]["paragraphs"])):
            contexts.append(data["data"][i]["paragraphs"][j]["context"])
            qas.append(data["data"][i]["paragraphs"][j]["qas"])
    return (contexts,qas)

In [6]:
CapPassage = False

from nltk.tokenize import word_tokenize
contexts,qas = extractor(data)

def find_sub_list(sl,l):
    sll=len(sl)
    for ind in (i for i,e in enumerate(l) if e==sl[0]):
        if l[ind:ind+sll]==sl:
            return ind,ind+sll
    return (-1,-1)

def capPassage(passage,answer,cap_length = 30):
    y = np.zeros(cap_length)
    left,right = find_sub_list(answer,passage)
    if(left==-1):
        return passage[0:cap_length]
    left = left - int((cap_length - len(answer))/2)
    right = right + int((cap_length + len(answer))/2)
    if(left < 0):
        left = 0
    if(right > len(passage)):
        right = len(passage)
    return passage[left:right]
    
def findAnsVec(answer,passage):
    ans = np.zeros((len(passage)))
    start,end = find_sub_list(answer,passage)
    if(start==-1):
        start = passage.index(answer[0])
        end = start + len(answer)
    ans[start:end] = 1
    return ans

In [7]:
X_train_comp_all = []
X_train_comp_ans_all = []
X_train_ans_all = []
Y_train_ques_all = []
invalid = 0
X_train_ans_label_all = []
for i,context in enumerate(contexts):
    passage = word_tokenize(context.lower())
    
    a_lab = np.zeros(len(passage))
    for j,_ in enumerate(qas[i]):
        answer = word_tokenize(qas[i][j]["answers"][0]['text'].lower())
        start,end = find_sub_list(answer,passage)
        if start == -1:
            continue
        a_lab[start:end+1] = 1
            
            
    for j,_ in enumerate(qas[i]):
        try:
            question = word_tokenize(qas[i][j]['question'].lower())
            answer = word_tokenize(qas[i][j]["answers"][0]['text'].lower())
            
            if CapPassage:
                cappedPassage = capPassage(passage,answer)
            else:
                cappedPassage = passage
            
            X_train_comp_ans_all.append(findAnsVec(answer,passage))
            X_train_ans_label_all.append(a_lab)
            X_train_comp_all.append(cappedPassage)
            X_train_ans_all.append(answer)
            Y_train_ques_all.append(question)
        except Exception as e:
            invalid = invalid+1
    

In [8]:
from collections import Counter
import operator
def findKMostFrequentWords(k):
    ctr = Counter([item for sublist in X_train_comp_all for item in sublist] + [item for sublist in Y_train_ques_all for item in sublist])
    sorted_ctr = sorted(ctr.items(), key=operator.itemgetter(1), reverse=True)
    return [item[0] for item in sorted_ctr[0:k]]

In [9]:
wordToTake = 2000
words = findKMostFrequentWords(100000)

In [10]:
_word_to_idx_reduced = {}
_idx_to_word_reduced = []


def _add_word_reduced(word):
    idx = len(_idx_to_word_reduced)
    _word_to_idx_reduced[word] = idx
    _idx_to_word_reduced.append(word)
    return idx


UNKNOWN_TOKEN = _add_word_reduced(UNKNOWN_WORD)
START_TOKEN = _add_word_reduced(START_WORD)
END_TOKEN = _add_word_reduced(END_WORD)




dimensions = glove.shape[1]
reduced_glove = []
reduced_glove.append(np.zeros(dimensions))
reduced_glove.append(-np.ones(dimensions))
reduced_glove.append(np.ones(dimensions))

for word in words:
    l = look_up_word(word)
    if(l != UNKNOWN_TOKEN):
        idx = _add_word_reduced(word)
        reduced_glove.append(glove[l])
        if(len(reduced_glove) == wordToTake):
            break
        
def look_up_word_reduced(word):
    return _word_to_idx_reduced.get(word, UNKNOWN_TOKEN)


def look_up_token_reduced(token):
    return _idx_to_word_reduced[token]

reduced_glove = np.array(reduced_glove)
reduced_glove.shape

(2000, 300)

In [11]:
!hostname
print(invalid)
for i in np.where(X_train_ans_label_all[110] == 1)[0]:
    print(X_train_comp_all[110][i])

cuda2.cims.nyu.edu
1493
rev
.
john
j.
cavanaugh
,
c.s.c
.
served
more
than
half
,
lobund
institute
for
animal
studies
and
medieval
institute
.
hall
of
liberal
arts
(


In [12]:
print(X_train_comp_all[0])
print(X_train_ans_all[0])

['architecturally', ',', 'the', 'school', 'has', 'a', 'catholic', 'character', '.', 'atop', 'the', 'main', 'building', "'s", 'gold', 'dome', 'is', 'a', 'golden', 'statue', 'of', 'the', 'virgin', 'mary', '.', 'immediately', 'in', 'front', 'of', 'the', 'main', 'building', 'and', 'facing', 'it', ',', 'is', 'a', 'copper', 'statue', 'of', 'christ', 'with', 'arms', 'upraised', 'with', 'the', 'legend', '``', 'ad', 'me', 'omnes', "''", '.', 'next', 'to', 'the', 'main', 'building', 'is', 'the', 'basilica', 'of', 'the', 'sacred', 'heart', '.', 'immediately', 'behind', 'the', 'basilica', 'is', 'the', 'grotto', ',', 'a', 'marian', 'place', 'of', 'prayer', 'and', 'reflection', '.', 'it', 'is', 'a', 'replica', 'of', 'the', 'grotto', 'at', 'lourdes', ',', 'france', 'where', 'the', 'virgin', 'mary', 'reputedly', 'appeared', 'to', 'saint', 'bernadette', 'soubirous', 'in', '1858.', 'at', 'the', 'end', 'of', 'the', 'main', 'drive', '(', 'and', 'in', 'a', 'direct', 'line', 'that', 'connects', 'through', '

In [13]:
find_sub_list(X_train_ans_all[0] , X_train_comp_all[0])

(101, 104)

In [14]:
print(invalid)
print(X_train_comp_all[101])
print(X_train_ans_all[101])
print(Y_train_ques_all[101])

c = list(zip(X_train_comp_all,X_train_comp_ans_all, X_train_ans_all, X_train_ans_label_all,Y_train_ques_all))
np.random.shuffle(c)
X_train_comp_all_shuffled,X_train_comp_ans_all_shuffled, X_train_ans_shuffled, X_train_ans_label_shuffled,Y_train_ques_all_shuffled = zip(*c)

print(X_train_comp_all_shuffled[101])
print(X_train_comp_ans_all_shuffled[101])
print(X_train_ans_shuffled[101])
print(X_train_ans_label_shuffled[101])
print(Y_train_ques_all_shuffled[101])


1493
['the', 'success', 'of', 'its', 'football', 'team', 'made', 'notre', 'dame', 'a', 'household', 'name', '.', 'the', 'success', 'of', 'note', 'dame', 'reflected', 'rising', 'status', 'of', 'irish', 'americans', 'and', 'catholics', 'in', 'the', '1920s', '.', 'catholics', 'rallied', 'up', 'around', 'the', 'team', 'and', 'listen', 'to', 'the', 'games', 'on', 'the', 'radio', ',', 'especially', 'when', 'it', 'knocked', 'off', 'the', 'schools', 'that', 'symbolized', 'the', 'protestant', 'establishment', 'in', 'america', '—', 'harvard', ',', 'yale', ',', 'princeton', ',', 'and', 'army', '.', 'yet', 'this', 'role', 'as', 'high-profile', 'flagship', 'institution', 'of', 'catholicism', 'made', 'it', 'an', 'easy', 'target', 'of', 'anti-catholicism', '.', 'the', 'most', 'remarkable', 'episode', 'of', 'violence', 'was', 'the', 'clash', 'between', 'notre', 'dame', 'students', 'and', 'the', 'ku', 'klux', 'klan', 'in', '1924.', 'nativism', 'and', 'anti-catholicism', ',', 'especially', 'when', 'dire

In [39]:
examples_to_take_train = 1000

X_train_comp = X_train_comp_all_shuffled[0:examples_to_take_train]
X_train_comp_ans = X_train_comp_ans_all_shuffled[0:examples_to_take_train]
X_train_ans = X_train_ans_shuffled[0:examples_to_take_train]
X_train_ans_label = X_train_ans_label_shuffled[0:examples_to_take_train]
Y_train_ques = Y_train_ques_all_shuffled[0:examples_to_take_train]
answer_indices = [np.where(x==1)[0].tolist() for x in X_train_comp_ans]

In [40]:
max_document_len = len(max(X_train_comp,key=len))
max_answer_len = len(max(X_train_ans,key=len))
max_question_len = len(max(Y_train_ques,key=len)) + 1

In [41]:
document_tokens = np.zeros((examples_to_take_train, max_document_len), dtype=np.int32)
document_lengths = np.zeros(examples_to_take_train, dtype=np.int32)
answer_labels = np.zeros((examples_to_take_train, max_document_len), dtype=np.int32)
answer_masks = np.zeros((examples_to_take_train, max_answer_len, max_document_len), dtype=np.int32)
answer_lengths = np.zeros(examples_to_take_train, dtype=np.int32)
question_input_tokens = np.full((examples_to_take_train, max_question_len), END_TOKEN, dtype=np.int32)
question_output_tokens = np.full((examples_to_take_train, max_question_len), END_TOKEN, dtype=np.int32)
question_lengths = np.zeros(examples_to_take_train, dtype=np.int32)
suppression_answer = np.zeros((examples_to_take_train, reduced_glove.shape[0], 1),dtype=np.int32)
expression_contexts = np.zeros((examples_to_take_train, max_question_len,reduced_glove.shape[0]),dtype=np.int32)
expression_probabilities = np.zeros((examples_to_take_train, max_question_len,reduced_glove.shape[0]),dtype=np.float32)

In [42]:
answer_labels[0]

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,

In [43]:
print(answer_labels.shape)
for i in range(examples_to_take_train):
    answer_labels[i,0:len(X_train_ans_label[i])] = X_train_ans_label[i]
    for j, word in enumerate(X_train_comp[i]):
        document_tokens[i, j] = look_up_word_reduced(word)
    document_lengths[i] = len(X_train_comp[i])

    for j, index in enumerate(answer_indices[i]):
        answer_masks[i, j, index] = 1
    answer_lengths[i] = len(answer_indices[i])
    
    #print(Y_train_ques[i])
    question_input_words = ([START_WORD] + Y_train_ques[i])
    question_output_words = (Y_train_ques[i] + [END_WORD])

    for j, word in enumerate(question_input_words):
            question_input_tokens[i, j] = look_up_word_reduced(word)
    for j, word in enumerate(question_output_words):
        question_output_tokens[i, j] = look_up_word_reduced(word)
    question_lengths[i] = len(question_input_words)
    
    for j, word in enumerate(X_train_ans[i]):
        if(word not in Y_train_ques[i]):
            suppression_answer[i, look_up_word_reduced(word),:] = 1
            
    words_to_consider_expression = set(X_train_comp[i] + nltkStopWords + punctuations)

    for j,word in enumerate(words_to_consider_expression):
        expression_contexts[i,:,look_up_word_reduced(word)] = 1
        
    for j,word in enumerate(words_to_consider_expression):
        expression_probabilities[i,:,look_up_word_reduced(word)] = len(np.where(expression_contexts[i][0] == 1)[0]) / float(wordToTake)
    expression_probabilities[i,:,np.where(expression_probabilities[i][0] == 0)[0]] = len(np.where(expression_contexts[i][0] == 0)[0]) / float(wordToTake)
    
        

(1000, 638)


In [44]:
words_to_consider_expression

{"''",
 ',',
 '-',
 '.',
 '13',
 '2011',
 '2015',
 '70',
 '75',
 '80',
 '?',
 '[',
 ']',
 '``',
 'a',
 'about',
 'above',
 'after',
 'again',
 'against',
 'ain',
 'all',
 'am',
 'american',
 'an',
 'and',
 'any',
 'are',
 'aren',
 "aren't",
 'as',
 'at',
 'be',
 'because',
 'been',
 'before',
 'being',
 'below',
 'between',
 'bike',
 'boston',
 'both',
 'but',
 'by',
 'can',
 'cities',
 'city',
 'commuters',
 'commutes',
 'compactness',
 'comparably',
 'couldn',
 "couldn't",
 'country',
 'd',
 'did',
 'didn',
 "didn't",
 'do',
 'does',
 'doesn',
 "doesn't",
 'doing',
 'don',
 "don't",
 'down',
 'during',
 'each',
 'factors',
 'few',
 'foot',
 'for',
 'from',
 'further',
 'had',
 'hadn',
 "hadn't",
 'has',
 'hasn',
 "hasn't",
 'have',
 'haven',
 "haven't",
 'having',
 'he',
 'her',
 'here',
 'hers',
 'herself',
 'highest',
 'him',
 'himself',
 'his',
 'hosts',
 'how',
 'i',
 'if',
 'in',
 'into',
 'is',
 'isn',
 "isn't",
 'it',
 "it's",
 'its',
 'itself',
 'just',
 'large',
 'll',
 'm',

In [45]:
print(len(np.where(expression_contexts[10][0] == 0)[0]))
print(len(np.where(expression_contexts[10][0] == 1)[0]))

print(len(np.where(expression_probabilities[10][0] > 0.5)[0]))
print(len(np.where(expression_probabilities[10][0] < 0.5)[0]))

1826
174
1826
174


In [46]:
def create_vocabulary(data):
    flat_list = [item for sublist in data for item in sublist]
    vocabulary = sorted(set(flat_list))
    vocabulary.append("<UNK>")
    vocabulary.append("unk")
    vocabulary.append("eos")
    vocabulary = ["<EOS>"] + vocabulary
    word_to_index = { word:i for i,word in enumerate(vocabulary) }
    index_to_word = { i:word for i,word in enumerate(vocabulary) }
    return (vocabulary,word_to_index,index_to_word)

In [47]:
print(reduced_glove.shape)
vocabulary_comp,word_to_index_comp,index_to_word_comp = create_vocabulary(X_train_comp + Y_train_ques)
print(len(vocabulary_comp))
print(word_to_index_comp["?"])
print(word_to_index_comp["what"])

(2000, 300)
17719
1299
17273


In [48]:
def create_one_hot_vector(data,vocabulary,word_to_index,index_to_word, maxLen):
    one_hot = np.zeros([maxLen,len(vocabulary)])
    for i,word in enumerate(data):
        if i >= maxLen:
            break
        if(word not in word_to_index):
            word = "<UNK>"
        one_hot[i][word_to_index[word]] = 1
    return one_hot

def create_one_hot_vector_from_indices(data,maxLen,vocabulary):
    one_hot = np.zeros([maxLen,len(vocabulary)])
    for i,indice in enumerate(data):
        if i >= maxLen:
            break
        one_hot[i][int(indice)] = 1
    return one_hot


def create_one_hot_training_Set(data,maxLen,vocabulary):
    one_hot_data = np.zeros([data.shape[0],maxLen,len(vocabulary)])
    for i in range(data.shape[0]):
        one_hot_data[i] = create_one_hot_vector_from_indices(data[i],maxLen,vocabulary)
    return one_hot_data




In [49]:
def context_to_indices_glove(X,max_len):
    
    m = len(X)                                 
    
    X_indices = np.full([m,max_len],look_up_word_reduced(END_WORD))
    
    for i in range(m):
        j = 0
        for w in X[i]:
            if(j>=max_len):
                break;
            
            X_indices[i, j] = look_up_word_reduced(w)
            j = j+1
    return X_indices

In [50]:
document_tokens = context_to_indices_glove(X_train_comp, max_document_len)

In [51]:
document_tokens[0]

array([   3,  386,  544, 1043,   43,   42,    0,    8,    3,    0,  226,
          0,    4,   32,    0, 1043,   45,    3,   47,    4,    3,    0,
          0,    4,   12,  333,    8,    0,   57,    0,    8,    3,  105,
        472,   83,    4,   20,  849,    0,    0,   15,    0,   14,    4,
          0,    0,    0,   15,    0,   14,    4,    7,    0,    0, 1042,
       1152,   15,    0,   14,    6,    0,  576,   39,    0,    8,   35,
       1898,    0,    0,    6,    8,    3,    0,    4,   20,    3,    0,
          5,    3,    0,    8,  386,    7,    3,  436,  115,    5,    0,
         57,  532,    4,   57, 1155,    8,  832,    4,    7,  160,    9,
         37,  145,    0, 1014,    0,  718,    4,    0,    0,    7, 1732,
          0,    4,    7,    0,    8,    0,    6,    2,    2,    2,    2,
          2,    2,    2,    2,    2,    2,    2,    2,    2,    2,    2,
          2,    2,    2,    2,    2,    2,    2,    2,    2,    2,    2,
          2,    2,    2,    2,    2,    2,    2,   

In [52]:
answer_labels.shape

(1000, 638)

## Batch Data Preparation

In [53]:
import math
def create_batch(inputs,batch_size,shuffle=False):
    num_batches = len(inputs[0]) // batch_size + 1
    outputs = []
    for index,inp in enumerate(inputs):
    
        output = {'document_tokens':[],
                    'document_lengths':[],
                    'answer_labels':[],
                    'answer_masks': [],
                    'answer_lengths': [],
                    'question_input_tokens':[],
                    'question_output_tokens':[],
                    'question_lengths':[],
                    'suppression_answer':[],
                    'expression_contexts': [],
                    'expression_probabilities':[]}
    
        start = 0
        for i in range(num_batches):
            if i == num_batches - 1:
                end = None
            else:
                end = start+batch_size
            maxD = max(inputs[1][start:end])
            maxA = max(inputs[4][start:end])
            maxQ = max(inputs[7][start:end])
            if index == 0:
                outputs['document_tokens'] = inp[start:end,:maxD]
            elif index==1:
                outputs['document_lengths'] = inp[start:end]
            elif index == 2:
                outputs['answer_labels']=inp[start:end,:maxD]
            elif index==3:
                outputs['answer_masks']=inp[start:end,:maxA,:maxD]
            elif index==4:
                outputs['answer_lengths']=inp[start:end]
            elif index==5:
                output['question_input_tokens']=inp[start:end,:maxQ]
            elif index==6:
                output['question_output_tokens']=inp[start:end,:maxQ]
            elif index==7:
                output['question_lengths'] = inp[start:end]
            elif index==8:
                output['suppression_answer'] = inp[start:end]
            elif index==9:
                output['expression_contexts'] = inp[start:end,0:maxQ,:]
            elif index==10: 
                output['expression_probabilities'] = inp[start:end,0:maxQ,:]
            start = start + batch_size
        outputs.append(output)
    
    return outputs

def createBatch(inputs,batch_size,shuffle=False):
    outputs = []
    num_batches = math.ceil(len(inputs[0])/batch_size)
    
    for index,inp in enumerate(inputs):
        start = 0
        output = []
        for i in range(num_batches-1):
            maxD = max(inputs[1][start:start+batch_size])
            maxA = max(inputs[4][start:start+batch_size])
            maxQ = max(inputs[7][start:start+batch_size])
            if index == 0 or index == 2:
                output.append(inp[start:start+batch_size,0:maxD]) 
            elif index==3:
                output.append(inp[start:start+batch_size,0:maxA,0:maxD]) 
            elif index==5 or index==6:
                output.append(inp[start:start+batch_size,0:maxQ])
            elif index==9 or index==10:
                output.append(inp[start:start+batch_size,0:maxQ,:])
            else: 
                output.append(inp[start:start+batch_size])
            start = start + batch_size
        
        # Remaining training sample i.e. training mod batch_size
        maxD = max(inputs[1][start:])
        maxA = max(inputs[4][start:])
        maxQ = max(inputs[7][start:])
        if index == 0 or index == 2:
            output.append(inp[start:,0:maxD]) 
        elif index==3:
            output.append(inp[start:,0:maxA,0:maxD]) 
        elif index==5 or index==6:
            output.append(inp[start:,0:maxQ])
        elif index==9 or index==10:
            output.append(inp[start:,0:maxQ,:]) 
        else: 
            output.append(inp[start:])
        outputs.append(output)
    
    return outputs


In [54]:
batch_size = 32
batch_input = createBatch([document_tokens,document_lengths,answer_labels,answer_masks,answer_lengths,question_input_tokens,question_output_tokens,question_lengths,suppression_answer,expression_contexts,expression_probabilities]
                    ,batch_size)
number_of_batches = len(batch_input[0])
print("Number of batches = ", number_of_batches)

Number of batches =  32


## Model

In [55]:
import torch
import torch.nn as nn
from torch.autograd import Variable
from torch import optim
import torch.nn.functional as F

use_cuda = torch.cuda.is_available()

### Document Embedding

In [56]:
class Embedder(nn.Module):
    def __init__(self, input_size, output_size):
        super(Embedder, self).__init__()
        self.embedding = nn.Embedding(input_size, output_size)
        
        # TODO: Verify
        self.embedding.weight = nn.Parameter(torch.from_numpy(reduced_glove).float())
        self.embedding.weight.requires_grad = False
    def forward(self, x):
        return self.embedding(x)

In [58]:
class AnswerEncoderRNN(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers=1):
        super(AnswerEncoderRNN, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        
        
        self.gru = nn.GRU(input_size, hidden_size, batch_first= True, bidirectional=True) #Input_size = Hidden_Size
        if use_cuda:
            self.gru = self.gru.cuda()
        self.fc = nn.Linear(hidden_size*2, 1)
        if use_cuda:
            self.fc = self.fc.cuda()

    def forward(self, x, hidden):
        output, hidden = self.gru(x, hidden)
        final_output = self.fc(output)
        final_output = F.sigmoid(final_output)
        self.hiddenState = hidden
        return final_output, output, hidden
    
    def initHidden(self):
        result = Variable(torch.zeros(2, batch_size, self.hidden_size)) #2 for BiDirectional
        if use_cuda:
            result = result.cuda()
        return result

In [59]:
class QuestionEncoderRNN(nn.Module):
    
    def __init__(self,input_size, hidden_size):
        super(QuestionEncoderRNN, self).__init__()
        self.hidden_size = hidden_size
        self.gru = nn.GRU(input_size, hidden_size, batch_first= True)
        if use_cuda:
            self.gru = self.gru.cuda()
    
    def forward(self, x, hidden):
        output, hidden = self.gru(x, hidden)
        self.hiddenState = hidden
        return output, hidden
    
    def initHidden(self):
        #result = Variable(torch.zeros(1, batch_size, self.hidden_size))
        result = Variable(torch.zeros(1, 1, self.hidden_size))
        if use_cuda:
            result = result.cuda()
        return result

In [75]:
class QuestionDecoderRNN(nn.Module):
    
    def __init__(self,input_size, hidden_size):
        super(QuestionDecoderRNN, self).__init__()
        self.hidden_size = hidden_size
        self.gru = nn.GRU(input_size, hidden_size, batch_first= True)
        if use_cuda:
            self.gru = self.gru.cuda()
        
    def forward(self, x, hidden):
        output, hidden = self.gru(x, hidden)
        self.hiddenState = hidden
        return output, hidden
    
    def initHidden(self):
        #result = Variable(torch(1, batch_size, self.hidden_size))
        result = Variable(torch.zeros(1, 1, self.hidden_size))
        if use_cuda:
            result = result.cuda()
        return result
    
'''
class FCLayer(nn.Module):
    def __init__(self,input_size, output_size):
        super(FCLayer, self).__init__()
        self.fc = nn.Linear(input_size, output_size)
        
    def forward(self, x):
        return self.fc(x)
'''

class QuestionGenerationFC(nn.Module):
    def __init__(self,input_size, output_size):
        super(QuestionGenerationFC, self).__init__()
        self.fc = nn.Linear(input_size, output_size)
        if use_cuda:
            self.fc = self.fc.cuda()

    def forward(self, x):
        output = self.fc(x)
        output = F.log_softmax(output, dim=-1)
        return output
         

In [76]:
hidden_size = reduced_glove.shape[1]

embedder = Embedder(input_size = reduced_glove.shape[0], output_size = reduced_glove.shape[1])
#fcLayer = FCLayer(hidden_size, hidden_size)
answerEncoder = AnswerEncoderRNN(input_size = hidden_size, hidden_size=int(hidden_size/2))
questionEncoder = QuestionEncoderRNN(input_size=hidden_size, hidden_size=hidden_size)
questionDecoder = QuestionDecoderRNN(input_size=hidden_size, hidden_size=hidden_size)
questionGenerator = QuestionGenerationFC(input_size = hidden_size, output_size=reduced_glove.shape[0])

answerEncoder.train()
questionEncoder.train()
questionDecoder.train()
questionGenerator.train()

train_param = []

for model in [embedder, answerEncoder, questionEncoder, questionDecoder, questionGenerator]:
    train_param += [p for p in model.parameters() if p.requires_grad]

print("Number of trainable parameters = ", len(train_param))

optimizer = torch.optim.Adam(train_param, 0.0001)
criterion1 = nn.BCELoss()
#criterion2 = nn.CrossEntropyLoss()
criterion2 = nn.NLLLoss()
#criterion2 = nn.MultiLabelSoftMarginLoss()

Number of trainable parameters =  20


In [77]:
def repackage_hidden(h):
    """Wraps hidden states in new Variables, to detach them from their history."""
    if type(h) == Variable:
        return Variable(h.data)
    else:
        return tuple(repackage_hidden(v) for v in h)

In [79]:
verboseBatchPrinting = True
averageBatchLossPrinting = True

num_epochs = 100
answer_encoder_hidden = answerEncoder.initHidden()
question_encoder_hidden = questionEncoder.initHidden()
question_decoder_hidden = None
for epoch in range(1, num_epochs+1):
    avg_loss = 0
    for batch_num in range(len(batch_input[0])):

        current_batch_size = len(batch_input[0][batch_num])
        if current_batch_size != batch_size:
            continue


        maxDocLenForBatch = int(max(batch_input[1][batch_num]))
        mask = np.zeros((current_batch_size, maxDocLenForBatch))
        for i in range(current_batch_size):
            mask[i][0:batch_input[1][0][i]] = 1

        inp = Variable(torch.from_numpy(batch_input[0][batch_num]).long())

        labels = Variable(torch.from_numpy(batch_input[2][batch_num])).long()
        if use_cuda:
            labels = labels.cuda()

        optimizer.zero_grad()
        embedded_inp = embedder(inp).cuda()
        answer_encoder_hidden = repackage_hidden(answer_encoder_hidden)
        answer_tags, answer_outputs, answer_encoder_hidden = answerEncoder(embedded_inp, answer_encoder_hidden)


        if use_cuda:
            answer_outputs = answer_outputs.cuda()
            answer_tags = answer_tags.cuda()


        t_document_mask = Variable(torch.from_numpy(mask)).float()
        if use_cuda:
            t_document_mask = t_document_mask.cuda()
        outputs = torch.mul(answer_tags.squeeze(-1),t_document_mask)


        answer_loss = criterion1(outputs, labels.float())


        t_answer_mask = Variable(torch.from_numpy(batch_input[3][batch_num])).float()
        if use_cuda:
            t_answer_mask = t_answer_mask.cuda()

        question_encoder_input = torch.matmul(t_answer_mask, answer_outputs.float())
        question_encoder_hidden_batch = Variable(torch.zeros(1,current_batch_size,questionEncoder.hidden_size))
        if use_cuda:
            question_encoder_hidden_batch = question_encoder_hidden_batch.cuda()

        question_encoder_hidden = repackage_hidden(question_encoder_hidden)
        for i in range(current_batch_size):
            _ , question_encoder_hidden = questionEncoder(question_encoder_input[i:i+1,0:batch_input[4][batch_num][i],:], question_encoder_hidden)
            question_encoder_hidden_batch[:,i:i+1,:] = question_encoder_hidden

        if type(question_decoder_hidden) == Variable:
            question_decoder_hidden = repackage_hidden(question_decoder_hidden)
        question_loss = 0
        for i in range(current_batch_size):
            question_decoder_hidden = question_encoder_hidden_batch[:,i:i+1,:].clone()
            embedded_inputs = embedder(torch.from_numpy(batch_input[5][batch_num][i]).long()).cuda()
            output_labels = Variable(torch.from_numpy(batch_input[5][batch_num][i]).long())
            if use_cuda:
                output_labels = output_labels.cuda()

            for quesL in range(batch_input[7][batch_num][i]):
                decoder_output, question_decoder_hidden = questionDecoder(
                    embedded_inputs[quesL:quesL+1].unsqueeze(1),
                    question_decoder_hidden)

                final_output = questionGenerator(decoder_output)
                output_label = Variable(torch.zeros(1,2000))
                if use_cuda:
                    output_label = output_label.cuda()
                output_label[:,batch_input[5][batch_num][i][quesL]] = 1
                question_loss += criterion2(final_output.squeeze(0),
                                           output_labels[quesL:quesL+1])
                ##question_loss += criterion2(final_output.squeeze(0), output_label)


        net_loss = answer_loss + question_loss
        net_loss.backward(retain_graph=True)
        optimizer.step()


        avg_loss+= net_loss.data[0]
        if verboseBatchPrinting:
            print ('Batch: %d \t Epoch : %d\tNet Loss: %.4f \tAnswer Loss: %.4f \tQuestion Loss: %.4f'
                   %(batch_num, epoch, net_loss.data[0], answer_loss.data[0], question_loss.data[0]))



    if averageBatchLossPrinting:
        print('Average Loss after Epoch %d : %.4f'
                   %(epoch, avg_loss/number_of_batches))

Batch: 0 	 Epoch : 1	Net Loss: 1285.2201 	Answer Loss: 0.0935 	Question Loss: 1285.1266
Batch: 1 	 Epoch : 1	Net Loss: 1225.1450 	Answer Loss: 0.3488 	Question Loss: 1224.7961
Batch: 2 	 Epoch : 1	Net Loss: 1347.7788 	Answer Loss: 0.4700 	Question Loss: 1347.3088
Batch: 3 	 Epoch : 1	Net Loss: 1136.5720 	Answer Loss: 0.6469 	Question Loss: 1135.9250
Batch: 4 	 Epoch : 1	Net Loss: 1204.6637 	Answer Loss: 0.2880 	Question Loss: 1204.3757
Batch: 5 	 Epoch : 1	Net Loss: 1362.7119 	Answer Loss: 0.4569 	Question Loss: 1362.2550
Batch: 6 	 Epoch : 1	Net Loss: 1293.2693 	Answer Loss: 0.4847 	Question Loss: 1292.7845
Batch: 7 	 Epoch : 1	Net Loss: 1374.1400 	Answer Loss: 0.3017 	Question Loss: 1373.8383
Batch: 8 	 Epoch : 1	Net Loss: 1278.2900 	Answer Loss: 0.2638 	Question Loss: 1278.0262
Batch: 9 	 Epoch : 1	Net Loss: 1202.0615 	Answer Loss: 0.3475 	Question Loss: 1201.7140
Batch: 10 	 Epoch : 1	Net Loss: 1191.8167 	Answer Loss: 0.4037 	Question Loss: 1191.4130
Batch: 11 	 Epoch : 1	Net Loss:

Batch: 30 	 Epoch : 3	Net Loss: 1065.8584 	Answer Loss: 0.3260 	Question Loss: 1065.5323
Average Loss after Epoch 3 : 1114.1214
Batch: 0 	 Epoch : 4	Net Loss: 1136.0283 	Answer Loss: 0.0934 	Question Loss: 1135.9348
Batch: 1 	 Epoch : 4	Net Loss: 1073.0780 	Answer Loss: 0.3479 	Question Loss: 1072.7301
Batch: 2 	 Epoch : 4	Net Loss: 1183.1899 	Answer Loss: 0.4692 	Question Loss: 1182.7207
Batch: 3 	 Epoch : 4	Net Loss: 998.6674 	Answer Loss: 0.6463 	Question Loss: 998.0211
Batch: 4 	 Epoch : 4	Net Loss: 1068.6509 	Answer Loss: 0.2873 	Question Loss: 1068.3635
Batch: 5 	 Epoch : 4	Net Loss: 1214.4072 	Answer Loss: 0.4560 	Question Loss: 1213.9512
Batch: 6 	 Epoch : 4	Net Loss: 1159.8948 	Answer Loss: 0.4840 	Question Loss: 1159.4108
Batch: 7 	 Epoch : 4	Net Loss: 1220.9806 	Answer Loss: 0.3011 	Question Loss: 1220.6794
Batch: 8 	 Epoch : 4	Net Loss: 1137.1965 	Answer Loss: 0.2634 	Question Loss: 1136.9331
Batch: 9 	 Epoch : 4	Net Loss: 1056.8293 	Answer Loss: 0.3467 	Question Loss: 1056

Batch: 29 	 Epoch : 6	Net Loss: 1011.0238 	Answer Loss: 0.2967 	Question Loss: 1010.7271
Batch: 30 	 Epoch : 6	Net Loss: 954.2358 	Answer Loss: 0.3257 	Question Loss: 953.9101
Average Loss after Epoch 6 : 993.9863
Batch: 0 	 Epoch : 7	Net Loss: 1015.0531 	Answer Loss: 0.0935 	Question Loss: 1014.9596
Batch: 1 	 Epoch : 7	Net Loss: 950.4347 	Answer Loss: 0.3473 	Question Loss: 950.0873
Batch: 2 	 Epoch : 7	Net Loss: 1050.9348 	Answer Loss: 0.4687 	Question Loss: 1050.4662
Batch: 3 	 Epoch : 7	Net Loss: 890.7523 	Answer Loss: 0.6459 	Question Loss: 890.1064
Batch: 4 	 Epoch : 7	Net Loss: 958.2288 	Answer Loss: 0.2868 	Question Loss: 957.9420
Batch: 5 	 Epoch : 7	Net Loss: 1089.7615 	Answer Loss: 0.4554 	Question Loss: 1089.3060
Batch: 6 	 Epoch : 7	Net Loss: 1050.3077 	Answer Loss: 0.4836 	Question Loss: 1049.8242
Batch: 7 	 Epoch : 7	Net Loss: 1095.6746 	Answer Loss: 0.3007 	Question Loss: 1095.3738
Batch: 8 	 Epoch : 7	Net Loss: 1019.3719 	Answer Loss: 0.2631 	Question Loss: 1019.1088


Batch: 29 	 Epoch : 9	Net Loss: 911.7618 	Answer Loss: 0.2964 	Question Loss: 911.4654
Batch: 30 	 Epoch : 9	Net Loss: 860.2070 	Answer Loss: 0.3254 	Question Loss: 859.8816
Average Loss after Epoch 9 : 896.2604
Batch: 0 	 Epoch : 10	Net Loss: 915.3244 	Answer Loss: 0.0935 	Question Loss: 915.2308
Batch: 1 	 Epoch : 10	Net Loss: 850.5538 	Answer Loss: 0.3469 	Question Loss: 850.2069
Batch: 2 	 Epoch : 10	Net Loss: 942.1748 	Answer Loss: 0.4683 	Question Loss: 941.7065
Batch: 3 	 Epoch : 10	Net Loss: 802.3641 	Answer Loss: 0.6456 	Question Loss: 801.7186
Batch: 4 	 Epoch : 10	Net Loss: 866.1948 	Answer Loss: 0.2865 	Question Loss: 865.9083
Batch: 5 	 Epoch : 10	Net Loss: 985.8471 	Answer Loss: 0.4550 	Question Loss: 985.3921
Batch: 6 	 Epoch : 10	Net Loss: 956.7692 	Answer Loss: 0.4832 	Question Loss: 956.2860
Batch: 7 	 Epoch : 10	Net Loss: 991.1663 	Answer Loss: 0.3004 	Question Loss: 990.8658
Batch: 8 	 Epoch : 10	Net Loss: 921.7026 	Answer Loss: 0.2629 	Question Loss: 921.4398
Batch

Batch: 29 	 Epoch : 12	Net Loss: 826.7694 	Answer Loss: 0.2961 	Question Loss: 826.4733
Batch: 30 	 Epoch : 12	Net Loss: 778.0050 	Answer Loss: 0.3252 	Question Loss: 777.6798
Average Loss after Epoch 12 : 812.7691
Batch: 0 	 Epoch : 13	Net Loss: 828.7115 	Answer Loss: 0.0935 	Question Loss: 828.6180
Batch: 1 	 Epoch : 13	Net Loss: 765.1636 	Answer Loss: 0.3466 	Question Loss: 764.8170
Batch: 2 	 Epoch : 13	Net Loss: 848.8610 	Answer Loss: 0.4680 	Question Loss: 848.3929
Batch: 3 	 Epoch : 13	Net Loss: 725.2749 	Answer Loss: 0.6453 	Question Loss: 724.6296
Batch: 4 	 Epoch : 13	Net Loss: 786.0872 	Answer Loss: 0.2862 	Question Loss: 785.8010
Batch: 5 	 Epoch : 13	Net Loss: 895.7130 	Answer Loss: 0.4547 	Question Loss: 895.2583
Batch: 6 	 Epoch : 13	Net Loss: 874.0981 	Answer Loss: 0.4830 	Question Loss: 873.6151
Batch: 7 	 Epoch : 13	Net Loss: 899.5807 	Answer Loss: 0.3002 	Question Loss: 899.2806
Batch: 8 	 Epoch : 13	Net Loss: 838.0161 	Answer Loss: 0.2627 	Question Loss: 837.7533
Ba

...

Batch: 29 	 Epoch : 27	Net Loss: 513.3903 	Answer Loss: 0.2950 	Question Loss: 513.0953
Batch: 30 	 Epoch : 27	Net Loss: 472.4104 	Answer Loss: 0.3239 	Question Loss: 472.0865
Average Loss after Epoch 27 : 501.5410
Batch: 0 	 Epoch : 28	Net Loss: 503.5187 	Answer Loss: 0.0934 	Question Loss: 503.4253
Batch: 1 	 Epoch : 28	Net Loss: 450.2137 	Answer Loss: 0.3455 	Question Loss: 449.8682
Batch: 2 	 Epoch : 28	Net Loss: 521.8163 	Answer Loss: 0.4672 	Question Loss: 521.3491
Batch: 3 	 Epoch : 28	Net Loss: 422.8539 	Answer Loss: 0.6441 	Question Loss: 422.2098
Batch: 4 	 Epoch : 28	Net Loss: 481.0826 	Answer Loss: 0.2853 	Question Loss: 480.7972
Batch: 5 	 Epoch : 28	Net Loss: 559.9806 	Answer Loss: 0.4536 	Question Loss: 559.5270
Batch: 6 	 Epoch : 28	Net Loss: 554.2052 	Answer Loss: 0.4820 	Question Loss: 553.7232
Batch: 7 	 Epoch : 28	Net Loss: 548.9105 	Answer Loss: 0.2992 	Question Loss: 548.6113
Batch: 8 	 Epoch : 28	Net Loss: 527.8994 	Answer Loss: 0.2620 	Question Loss: 527.6374
Ba

Batch: 29 	 Epoch : 30	Net Loss: 463.5055 	Answer Loss: 0.2948 	Question Loss: 463.2107
Batch: 30 	 Epoch : 30	Net Loss: 425.3171 	Answer Loss: 0.3237 	Question Loss: 424.9934
Average Loss after Epoch 30 : 452.6463
Batch: 0 	 Epoch : 31	Net Loss: 452.6005 	Answer Loss: 0.0933 	Question Loss: 452.5072
Batch: 1 	 Epoch : 31	Net Loss: 402.9200 	Answer Loss: 0.3453 	Question Loss: 402.5747
Batch: 2 	 Epoch : 31	Net Loss: 472.8937 	Answer Loss: 0.4670 	Question Loss: 472.4266
Batch: 3 	 Epoch : 31	Net Loss: 375.4872 	Answer Loss: 0.6439 	Question Loss: 374.8433
Batch: 4 	 Epoch : 31	Net Loss: 432.6158 	Answer Loss: 0.2852 	Question Loss: 432.3306
Batch: 5 	 Epoch : 31	Net Loss: 507.6220 	Answer Loss: 0.4534 	Question Loss: 507.1686
Batch: 6 	 Epoch : 31	Net Loss: 502.4950 	Answer Loss: 0.4818 	Question Loss: 502.0132
Batch: 7 	 Epoch : 31	Net Loss: 493.6481 	Answer Loss: 0.2990 	Question Loss: 493.3491
Batch: 8 	 Epoch : 31	Net Loss: 478.4209 	Answer Loss: 0.2619 	Question Loss: 478.1590
Ba

Batch: 29 	 Epoch : 33	Net Loss: 416.5462 	Answer Loss: 0.2946 	Question Loss: 416.2516
Batch: 30 	 Epoch : 33	Net Loss: 381.7248 	Answer Loss: 0.3234 	Question Loss: 381.4013
Average Loss after Epoch 33 : 407.2000
Batch: 0 	 Epoch : 34	Net Loss: 405.3403 	Answer Loss: 0.0933 	Question Loss: 405.2471
Batch: 1 	 Epoch : 34	Net Loss: 359.7278 	Answer Loss: 0.3451 	Question Loss: 359.3827
Batch: 2 	 Epoch : 34	Net Loss: 427.4138 	Answer Loss: 0.4669 	Question Loss: 426.9469
Batch: 3 	 Epoch : 34	Net Loss: 332.2564 	Answer Loss: 0.6436 	Question Loss: 331.6127
Batch: 4 	 Epoch : 34	Net Loss: 387.5523 	Answer Loss: 0.2850 	Question Loss: 387.2673
Batch: 5 	 Epoch : 34	Net Loss: 458.9917 	Answer Loss: 0.4532 	Question Loss: 458.5385
Batch: 6 	 Epoch : 34	Net Loss: 454.1475 	Answer Loss: 0.4816 	Question Loss: 453.6659
Batch: 7 	 Epoch : 34	Net Loss: 442.4993 	Answer Loss: 0.2988 	Question Loss: 442.2005
Batch: 8 	 Epoch : 34	Net Loss: 432.4718 	Answer Loss: 0.2618 	Question Loss: 432.2100
Ba

Batch: 29 	 Epoch : 36	Net Loss: 372.5251 	Answer Loss: 0.2945 	Question Loss: 372.2306
Batch: 30 	 Epoch : 36	Net Loss: 341.5122 	Answer Loss: 0.3232 	Question Loss: 341.1890
Average Loss after Epoch 36 : 365.0214
Batch: 0 	 Epoch : 37	Net Loss: 361.4427 	Answer Loss: 0.0932 	Question Loss: 361.3495
Batch: 1 	 Epoch : 37	Net Loss: 320.2202 	Answer Loss: 0.3449 	Question Loss: 319.8752
Batch: 2 	 Epoch : 37	Net Loss: 385.0233 	Answer Loss: 0.4668 	Question Loss: 384.5565
Batch: 3 	 Epoch : 37	Net Loss: 293.0675 	Answer Loss: 0.6434 	Question Loss: 292.4241


KeyboardInterrupt: 

In [65]:
question_loss.data[0]

0.0

In [67]:
criterion2(final_output.squeeze(0), output_labels[quesL:quesL+1])

Variable containing:
 0
[torch.cuda.FloatTensor of size 1 (GPU 0)]

In [ ]:
embedded_inp.shape

In [81]:
checkpoint = torch.load("/data/ra2630/qgen_base_40k.pt")

NameError: name 'checkpoint' is not defined

In [ ]:
encodedAnswerOutputs = answerEncoder(embedded_inp[:,:,:], answerEncoder.initHidden())

In [ ]:
encodedAnswerOutputs[0][0]

In [ ]:

##########ONly Batch

verboseBatchPrinting = True
averageBatchLossPrinting = True

num_epochs = 100
answer_encoder_hidden = answerEncoder.initHidden()
question_encoder_hidden = questionEncoder.initHidden()
question_decoder_hidden = None
for epoch in range(1, num_epochs+1):
    avg_loss = 0
    for batch_num in range(len(batch_input[0])):
        
        current_batch_size = len(batch_input[0][batch_num])
        if current_batch_size != batch_size:
            continue
        
        
        maxDocLenForBatch = int(max(batch_input[1][batch_num]))
        mask = np.zeros((current_batch_size, maxDocLenForBatch))
        for i in range(current_batch_size):
            mask[i][0:batch_input[1][0][i]] = 1

            
        inp = Variable(torch.from_numpy(batch_input[0][batch_num]).long())

        labels = Variable(torch.from_numpy(batch_input[2][batch_num])).long()
        if use_cuda:
            labels = labels.cuda()

        ################################### Answer Encoder + Tagging    

        optimizer.zero_grad()
        embedded_inp = embedder(inp)
        if use_cuda:
            embedded_inp = embedder(inp).cuda()

            
        
        answer_encoder_hidden = repackage_hidden(answer_encoder_hidden)
        answer_tags, answer_outputs, answer_encoder_hidden = answerEncoder(embedded_inp, answer_encoder_hidden)
        

        
        
        if use_cuda:
            answer_outputs = answer_outputs.cuda()
            answer_tags = answer_tags.cuda()

        #answer_tags.requires_grad=False
        #answer_outputs.requires_grad=False
        
        
        t_document_mask = Variable(torch.from_numpy(mask)).float()
        if use_cuda:
            t_document_mask = t_document_mask.cuda()
        #t_document_mask.requires_grade = False
        
        outputs = answer_tags.squeeze(-1) * t_document_mask
        #outputs.requires_grad = False
        
        
        answer_loss = criterion1(outputs, labels.float())
        ################################### Q Encoder    
        
        t_answer_mask = Variable(torch.from_numpy(batch_input[3][batch_num])).float()
        if use_cuda:
            t_answer_mask = t_answer_mask.cuda()

        #t_answer_mask.requires_gradui = False
        
        # masking the non-answer embeddings
        question_encoder_input = torch.matmul(t_answer_mask, answer_outputs.float())
        #question_encoder_input.requires_grad = False
        question_encoder_output = Variable(torch.zeros(1,batch_size,questionEncoder.hidden_size))
        #question_encoder_output.requires_grad = False
        
        
        if use_cuda:
            question_encoder_hidden = question_encoder_hidden_batch.cuda()
            
        question_encoder_hidden = repackage_hidden(question_encoder_hidden)
        question_encoder_output, question_encoder_hidden = questionEncoder(question_encoder_input, question_encoder_hidden)
        
        #question_encoder_output.requires_grad = False

        
#         question_encoder_output = Variable(torch.zeros(batch_size, max_question_len, questionEncoder.hidden_size))
#         question_encoder_hidden = Variable(torch.zeros(batch_size, max_question_len, questionEncoder.hidden_size))
        
#         for i in range(current_batch_size):
#             question_encoder_output, question_encoder_hidden = questionEncoder(question_encoder_input[i:i+1,0:batch_input[4][batch_num][i],:], question_encoder_hidden)
#             question_encoder_hidden_batch[:,i:i+1,:] = question_encoder_hidden    

        ################################### Q Decoder    
        
        question_loss = 0
        
        question_decoder_hidden = question_encoder_hidden
        question_decoder_hidden = repackage_hidden(question_decoder_hidden)
        
        embedded_inputs = embedder(torch.from_numpy(batch_input[5][batch_num]).long())
        #embedded_inputs.requires_grad = False
        
        output_labels = Variable(torch.from_numpy(batch_input[6][batch_num]).long())
        
        decoder_output = Variable(torch.zeros(1,batch_size,questionDecoder.hidden_size))
        
        if use_cuda:
            embedded_inputs = embedded_inputs.cuda()
            output_labels = output_labels.cuda()
            
        
        question_decoder_hidden = repackage_hidden(question_decoder_hidden)
        for quesL in range(max(batch_input[7][batch_num])):
            decoder_output, question_decoder_hidden = questionDecoder(
                embedded_inputs[:,quesL:quesL+1,:],
                question_decoder_hidden)

            final_output = questionGenerator(decoder_output)

            output_label = Variable(torch.zeros(32,2000))
            if use_cuda:
                output_label = output_label.cuda()
            for b in range(len(batch_input[5][0])):
                output_label[b,batch_input[5][batch_num][b][quesL]] = 1
            #question_loss += criterion2(final_output.squeeze(1), 
            #                           output_labels[:,quesL:quesL+1])
            question_loss += criterion2(final_output.squeeze(0), output_label)
        
    
        
    
        net_loss = answer_loss + question_loss
        net_loss.backward()
    
        optimizer.step()
        
        avg_loss+= net_loss.data[0]
        if verboseBatchPrinting:
            print ('Batch: %d \t Epoch : %d\tNet Loss: %.4f \tAnswer Loss: %.4f \tQuestion Loss: %.4f' 
                   %(batch_num, epoch, net_loss.data[0], answer_loss.data[0], question_loss.data[0]))
            
    torch.cuda.empty_cache()
          
        
    if averageBatchLossPrinting:
        print('Average Loss after Epoch %d : %.4f'
                   %(epoch, avg_loss/number_of_batches))

In [ ]:
question_encoder_output.shape

In [ ]:
question_encoder_input.size()

In [ ]:
question_encoder_hidden.size()

In [ ]:
import gc
sss = None
for obj in gc.get_objects():
    try:
        if torch.is_tensor(obj) or (hasattr(obj, 'data') and torch.is_tensor(obj.data)):
            print(type(obj), obj.size())
            sss=obj
    except:
        None

In [ ]:
answer_tags.squeeze(-1).size()

In [ ]:
t_document_mask.size()

In [ ]:
# NLL Loss
question_mask = tf.sequence_mask(decoder_lengths ,dtype=tf.float32)
question_loss = seq2seq.sequence_loss(
    logits=decoder_outputs, targets=decoder_labels, weights=question_mask,
    name="question_loss")
question_loss = tf.Print(question_loss, [question_loss], message="question_loss: ")

#Suppression Loss
lambdaSuppress = 1

suppression_loss = lambdaSuppress * tf.reduce_sum(tf.matmul(tf.nn.softmax(decoder_outputs), s_answer))
suppression_loss = tf.Print(suppression_loss, [suppression_loss], message="suppression_loss: ")


In [ ]:
#Expression Loss

express_loss_f = tf.reduce_sum(-tf.log(tf.multiply(tf.sigmoid(decoder_outputs),e_context) + 10e-7))
suppress_loss_f = tf.reduce_sum(-tf.log(tf.multiply((1 - tf.sigmoid(decoder_outputs)),(1 - e_context)) + 10e-7))
expression_loss = (express_loss_f + suppress_loss_f)/(32 * 20 * wordToTake)
expression_loss = tf.Print(expression_loss, [expression_loss], message="expression_loss: ")



In [ ]:
x = tf.stack([question_loss,answer_loss,suppression_loss,expression_loss])
loss = tf.reduce_sum(tf.multiply(x, lossFlags))

In [ ]:
def shuffle_list(*ls):
    l =list(zip(*ls))
    np.random.shuffle(l)
    return zip(*l)

In [ ]:
expression_contexts.shape

In [ ]:
print("No of features:",len( batch_input))
print("No of batches:",len( batch_input[0]))

In [ ]:
optimizer = tf.train.AdamOptimizer(learning_rate=3e-3).minimize(loss)

saver = tf.train.Saver()
config = tf.ConfigProto()
config.gpu_options.allow_growth = True
session = tf.InteractiveSession(config=config)

session.run(tf.global_variables_initializer())
# session.run(tf.variables_initializer(saved_vars))


In [ ]:
EPOCHS = 400
loss_flag = np.array([1,1,1,1])

import time

for epoch in range(1, EPOCHS + 1):
    batch_loss = 0
    print("Epoch {0}".format(epoch))
    start_time = time.time()
    for batchNum in range(len(batch_input[0])):
        print("Batch : ",batchNum)
        t = session.run([optimizer, loss, question_loss, answer_loss, suppression_loss, expression_loss], {
            d_tokens: batch_input[0][batchNum],
            d_lengths: batch_input[1][batchNum],
            a_labels: batch_input[2][batchNum],
            encoder_input_mask: batch_input[3][batchNum],
            encoder_lengths: batch_input[4][batchNum],
            decoder_inputs: batch_input[5][batchNum],
            decoder_labels: batch_input[6][batchNum],
            decoder_lengths: batch_input[7][batchNum],
            s_answer: batch_input[8][batchNum],
            e_context: batch_input[9][batchNum],
            e_probs: batch_input[10][batchNum],
            lossFlags : loss_flag,
        })
        print("Loss: {0}".format(t[1]))
        batch_loss += t[2]
    batch_loss /= len(batch_input[0])
    end_time = time.time()
    print("Average Batch Question Loss: {0}".format(batch_loss))
    print("Time taken to complete epoch : " , (end_time-start_time)/60 , " minutes")
    if batch_loss < minQuestionLoss:
        print("Turning on all losses")
        #loss_flag = np.array([1,1,1,1])
    if(epoch%5 == 0):
        print("Saving model")
        #saver.save(session, "qgen-model")

In [ ]:
reduced_glove.shape[0]

In [ ]:
saver = tf.train.Saver()
saver.save(session, "qgen-model")

In [ ]:
batch_input[5][0].shape

In [ ]:
config = tf.ConfigProto()
config.gpu_options.allow_growth = True
session = tf.Session(config=config)
saver = tf.train.Saver()

saver.restore(session, 'qgen-model')

In [ ]:
answers = session.run(answer_tags, {
    d_tokens: batch_input[0][0],
    d_lengths: batch_input[1][0],
})
print(answers.shape)
print(answers[0])
answers = np.argmax(answers, 2)
print(answers.shape)
print(answers[0])

In [ ]:
for i in range(276):
    print("Prediction")
    printAllAns(answers,2,0)
    print("Ground Truth")
    printAllAns(batch_input[2][2],2,0)

In [ ]:
def printDoc(batch,num):
    for i in batch_input[0][batch][num]:
        print(look_up_token_reduced(i),sep=" ", end=" ")
    print(" ")

def printQues(batch,num):
    for i in batch_input[5][batch][num]:
        print(look_up_token_reduced(i),sep=" ", end=" ")
    print(" ")
    
def printAnsForQuestion(batch, num):
    for i in batch_input[5][batch][num]:
        print(look_up_token_reduced(i),sep=" ", end=" ")
    print(" ")
    
def printAllAns(answers, batch, num):
    for i,word in enumerate(batch_input[0][batch][num]):
        if answers[num][i] == 1 :
            print(look_up_token_reduced(word),sep=" ", end=" ")
    print(" ")
    

In [ ]:
import itertools

batchNum = 0

helper = seq2seq.GreedyEmbeddingHelper(embedding, tf.fill([batch_input[0][batchNum].shape[0]], START_TOKEN), END_TOKEN)
decoder = seq2seq.BasicDecoder(decoder_cell, helper, encoder_state, output_layer=projection)
decoder_outputs, _, _ = seq2seq.dynamic_decode(decoder, maximum_iterations=max_question_len)
decoder_outputs = decoder_outputs.rnn_output


questions = session.run(decoder_outputs, {
    d_tokens: batch_input[0][batchNum],
    d_lengths: batch_input[1][batchNum],
    a_labels: batch_input[2][batchNum],
    encoder_input_mask: batch_input[3][batchNum],
    encoder_lengths: batch_input[4][batchNum],
    e_context: batch_input[9][batchNum],
})



In [ ]:
batch_input[9][batchNum].shape

In [ ]:
#questions[:,:,END_TOKEN] = 0
qs = np.argmax(questions, 2)

In [ ]:
def sigmoid(x):
    return 1.0 / (1.0 + np.exp(-x))

In [ ]:

for i in np.argsort(questions[0][0])[-100:]:
    print(sigmoid(questions[0][0][i]))
    

In [ ]:
for i in np.argsort(questions[0][10])[-100:]:
    print(sigmoid(questions[0][10][i]),look_up_token_reduced(i), sep=" ", end= " ")
print("")
print(X_train_comp[0])

In [ ]:
p1 = 0
p2 = 0

for i in np.where(batch_input[9][0][0][0] == 1)[0]:
    p1 += -np.log(sigmoid(questions[0][0][i]) + 10e-7)
    
for i in np.where(batch_input[9][0][0][0] == 0)[0]:
    p2 += -np.log(sigmoid(questions[0][0][i]) + 10e-7)

print(p1)
print(p2)
print(p2-p1)

In [ ]:
for i in range(batch_input[0][batchNum].shape[0]):
    print("---------------------------------------------------------------------------------------------")
    question = itertools.takewhile(lambda t: t != END_TOKEN, qs[i])
    print("Generated Question: " + " ".join(look_up_token_reduced(token) for token in question))
    print("Ground Truth Question: ")
    printQues(batchNum,i)
    print("Ground Truth Answer: ", X_train_ans_shuffled[batch_size*batchNum + i])
    print("Context:")
    printDoc(batchNum,i)
    print("---------------------------------------------------------------------------------------------")
    
    

In [ ]:
#def trainDecoder(batch_num, example_num, answer_encoder_hidden, question_encoder_hidden, question_decoder_hidden):

batch_num = 0
example_num = 0
    
document_token = batch_input[0][batch_num][example_num:example_num+1]
document_length = batch_input[1][batch_num][example_num]
answer_label = batch_input[2][batch_num][example_num:example_num+1]
answer_mask = batch_input[3][batch_num][example_num:example_num+1]
answer_length = batch_input[4][batch_num][example_num]
question_input_token = batch_input[5][batch_num][example_num:example_num+1]
question_output_token = batch_input[6][batch_num][example_num:example_num+1]
question_length = batch_input[7][batch_num][example_num:example_num+1]
suppression_answer = batch_input[8][batch_num][example_num:example_num+1]

mask = np.zeros(document_length)
mask[0:document_length] = 1

inp = Variable(torch.from_numpy(document_token).long())

labels = Variable(torch.from_numpy(answer_label)).long()
if use_cuda:
    labels = labels.cuda()

################################### Answer Encoder + Tagging    

optimizer.zero_grad()
embedded_inp = embedder(inp)
if use_cuda:
    embedded_inp = embedder(inp).cuda()

answer_tags, answer_outputs, answer_encoder_hidden = answerEncoder(embedded_inp, answer_encoder_hidden)


if use_cuda:
    answer_outputs = answer_outputs.cuda()
    answer_tags = answer_tags.cuda()


t_document_mask = Variable(torch.from_numpy(mask)).float()
if use_cuda:
    t_document_mask = t_document_mask.cuda()

outputs = answer_tags.squeeze(-1) * t_document_mask


answer_loss = criterion1(outputs, labels.float())
################################### Q Encoder    

t_answer_mask = Variable(torch.from_numpy(answer_mask)).float()
if use_cuda:
    t_answer_mask = t_answer_mask.cuda()

# masking the non-answer embeddings
question_encoder_input = torch.matmul(t_answer_mask, answer_outputs.float())
question_encoder_input = question_encoder_input[0:answer_length]

_, question_encoder_hidden = questionEncoder(question_encoder_input, question_encoder_hidden)


question_loss = 0

question_decoder_hidden = question_encoder_hidden
embedded_inputs = embedder(torch.from_numpy(question_input_token).long())
if use_cuda:
    embedded_inputs = embedded_inputs.cuda()
output_labels = Variable(torch.from_numpy(question_output_token).long())
if use_cuda:
    output_labels = output_labels.cuda()

for quesL in range(question_length):
    decoder_output, question_decoder_hidden = questionDecoder(
        embedded_inputs[quesL:quesL+1].unsqueeze(1),
        question_decoder_hidden)

    final_output = questionGenerator(decoder_output)

    output_label = Variable(torch.zeros(1,2000))
    if use_cuda:
        output_label = output_label.cuda()

    output_label[:, question_input_token[quesL]] = 1
    question_loss += criterion2(final_output.squeeze(0), 
                               output_labels[quesL:quesL+1])
    #question_loss += criterion2(final_output.squeeze(0), output_label)



net_loss = answer_loss + question_loss
net_loss.backward()
optimizer.step()
    
    #return answer_loss, question_loss, net_loss, answer_encoder_hidden, question_encoder_hidden, question_decoder_hidden

In [ ]:
if epoch % 2 == 0:
    print("ok")

In [ ]:
verboseBatchPrinting = True
averageBatchLossPrinting = True
backward_called = False

num_epochs = 100
answer_encoder_hidden = answerEncoder.initHidden()
question_encoder_hidden = questionEncoder.initHidden()
question_decoder_hidden = None
for epoch in range(1, num_epochs+1):
    avg_loss_per_epoch = 0
    for batch_num in range(len(batch_input[0])):
        for example in range(len(batch_input[0][0])):
            answer_loss, question_loss, net_loss, answer_encoder_hidden, question_encoder_hidden, question_decoder_hidden = \
            trainDecoder(batch_num, example, answer_encoder_hidden, question_encoder_hidden, question_decoder_hidden)
            if verboseBatchPrinting:
                print ('Batch: %d \t Example: %d \tEpoch : %d\tNet Loss: %.4f \tAnswer Loss: %.4f \tQuestion Loss: %.4f' 
                       %(batch_num, example, epoch, net_loss.data[0], answer_loss.data[0], question_loss.data[0]))
            avg_loss+= net_loss.data[0]
            
    if averageBatchLossPrinting:
        print('Average Loss after Epoch %d : %.4f'
                   %(epoch, avg_loss/examples_to_take_train))